In [1]:
%pip install word-search-generator
%pip install openai
%pip install random-word pyyaml
%pip install nltk


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     |████████████████████████████████| 1.5 MB 2.7 MB/s eta 0:00:01
     |████████████████████████████████| 772 kB 12.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [35]:
import openai
import word_search_generator
import random
import nltk
import numpy as np


In [108]:
# global constant variables

# output dir to save data files
OUTPUT_DIR = "./out"
# name of files to append to filename
TEMPLATE_FILE_PATH = "data"
# type of extension for data files
EXTENSION_FILE = "csv"

# error files dir path
OUTPUT_ERROR_DIR = "./out/error"
# name of error files to append to error filename
TEMPLATE_ERROR_FILE_PATH = "error"

# openai variables

TEMPERATURE = 0
# AVAILABLE MODELS [gpt-3.5-turbo-16k, gpt-4, gpt-4-0613]
MODEL = 'gpt-3.5-turbo-16k'

# config variables

# increase time in case of error *`ServiceUnavailableError: The server is overloaded or not ready yet.`*
REQ_INTERVAL = 5  # 10 sec
VARIABILITY_TRIES = 3
MAX_PUZZLES = 5

# debug flag
DEBUG = True


In [81]:
def remove_last_line_from_string(s):
    return [s[:s.rfind('\n')], s[s.rfind('\n'):]]
    
def formatBoard(string):
    # Split the string into lines
    lines = string.splitlines()

    # Remove the first 3 lines using slicing
    new_lines = lines[3:-3]
    # Join the remaining lines into a new string
    result = '\n'.join(new_lines)

    return result


In [36]:
openai.api_key = ""


def request(
    prompt="",
    context=np.array([]),
):
    """
    Helper function to make a request to openai api, and return it's response.
    args:
        prompt (string)
        model (str)
        temperature (int)
    returns:
        str
    """
    context = np.append(
        context,
        np.array([{
            "role": 'user',
            "content": f"""
                {prompt}
            """,
        }]), axis=0)

    response = openai.ChatCompletion.create(
        model=MODEL,
        messages=context.tolist(),
        temperature=TEMPERATURE,  # this is the degree of randomness of the model's output
    )

    return response.choices[0].message["content"]

In [97]:
context = np.array([])
messages = np.array([
    {
        "role": "system",
        "content": f"""      
                You are a word searcher player playing word seach puzzles. \
                Your main task will be to find and return the coordinate in which \
                you fount the target word.
                
                Therefore, I will provide you with the target word to find. Also \
                the word search board will be given to you where the word is localted. \
                Words can go SW, N, S, NE, NW, SE, E, and W.
                
                
                The prompt input structure will be something like this:
                
                ```
                    "target": <word>
                    "board" : <board>
                ```
                
                Your task is to return the direction and position if in which the target \
                word was found. An example of the result could be HEDEOMA S @ (10, 1), where HEDEOMA is 
                the target word to find, the S stands for south and the (10, 1), means column 10,
                row 1 of the word puzzle \
                
                The JSON would be something like this:
                
                  ```
                    "word": HEDEOMA
                    "direction": S
                    "row": 10
                    "column": 1
                ```

                Take into account that for both row and column indices the first one is index 1, \
                the second one is index 2...
                
                Please do not provide any extra definition or information, just output \
                the result in a JSON Object with the following format:
                
                ```
                    "word": <word>
                    "direction": <direciton>
                    "row": <row>
                    "column": <column>
                ```
            """,
    },
])

context = np.append(messages, context)

In [38]:
from nltk.corpus import words
nltk.download('brown')

[nltk_data] Downloading package brown to /home/balbert/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [105]:
# generate random word
word_list = words.words()  # get all English words
word = random.choice(word_list)  # select a random word

puzzle = word_search_generator.WordSearch(words=word)
puzzle.level = 1
# retrieve puzzle with no answer, and answer separated
[puzzle, answer] = remove_last_line_from_string(str(puzzle))


In [109]:
# generate prompt
prompt = f"""
    "target": {word}
    "board": { formatBoard(puzzle)}
"""

req = request(prompt=prompt, context=context).replace(" ", "")
print(req)
print(answer)

{"word":"wormless","direction":"SE","row":7,"column":1}

Answer Key: WORMLESS S @ (11, 4)
